<a href="https://colab.research.google.com/github/Grzegorz72/dw_matrix/blob/master/Day5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install eli5

     |████████████████████████████████| 112kB 3.0MB/s 


In [4]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score

import eli5
from eli5.sklearn import  PermutationImportance

from ast import literal_eval
from tqdm import tqdm_notebook

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)
Using TensorFlow backend.


In [5]:
cd "/content/drive/My Drive/Colab Notebooks/dw_matrix"

/content/drive/My Drive/Colab Notebooks/dw_matrix


In [6]:
ls

data/  HelloGithub.ipynb  LICENSE  matrix_one/  README.md


In [7]:
ls data

men_shoes.csv


In [0]:
df = pd.read_csv('data/men_shoes.csv', low_memory=False)

In [9]:
df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension', 'ean', 'features',
       'flavors', 'imageurls', 'isbn', 'keys', 'manufacturer',
       'manufacturernumber', 'merchants', 'name', 'prices_amountmin',
       'prices_amountmax', 'prices_availability', 'prices_color',
       'prices_condition', 'prices_count', 'prices_currency',
       'prices_dateadded', 'prices_dateseen', 'prices_flavor', 'prices_issale',
       'prices_merchant', 'prices_offer', 'prices_returnpolicy',
       'prices_shipping', 'prices_size', 'prices_source', 'prices_sourceurls',
       'prices_warranty', 'quantities', 'reviews', 'sizes', 'skus',
       'sourceurls', 'upc', 'vin', 'websiteids', 'weight'],
      dtype='object')

In [0]:
def run_model(feats, model = DecisionTreeRegressor(max_depth=80)):
  X = df[feats].values
  y = df['prices_amountmin'].values

  #model = DecisionTreeRegressor(max_depth=100)
  scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [11]:
df['brand_cat'] = df['brand'].map(lambda x: str(x).lower()).factorize()[0]
run_model(['brand_cat'])

(-45.84410217248452, 3.216343160242935)

In [12]:
model = RandomForestRegressor(max_depth=26, n_estimators=600, random_state=2019)
run_model(['brand_cat'], model)
#(-45.70030308619751, 3.2437088749710243)

(-45.599095554425695, 3.220267131985374)

In [13]:
df.features.head(20).values

array(['[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SERVUS BY HONEYWELL"]},{"key":"manufacturer_part_number","value":["ZSR101BLMLG"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SER

In [0]:
def parse_features(x):
  output_dict = {}
  if str(x) == 'nan': return output_dict
  features = literal_eval(x.replace('\\"','"'))
  
  for item in features:
    key = item['key'].lower().strip()
    value = item['value'][0].lower().strip()
    output_dict[key] = value
  return output_dict


df['parsed_features'] = df['features'].map(parse_features)

In [15]:
df['parsed_features'][100]

{'color': 'brown',
 'condition': 'new with tags',
 'ean': '0711830229263',
 'style': 'messenger/shoulder bag'}

In [16]:
keys =set()

df['parsed_features'].map(lambda x: keys.update(x.keys()))
len(keys)

476

In [17]:
def get_name_feat(key):
  return 'feat_' + key

for key in tqdm_notebook(keys):
    df[get_name_feat(key)] = df.parsed_features.map(lambda feats: feats[key] if key in feats else np.nan)

In [18]:
keys_stat = {}
for key in tqdm_notebook(keys):
  keys_stat[key] = df[False == df[get_name_feat(key)].isnull()].shape[0] / df.shape[0] * 100

In [19]:
{k:v for k,v in keys_stat.items() if v>20}

{'age group': 27.64770240700219,
 'brand': 48.62691466083151,
 'color': 47.784463894967175,
 'gender': 50.17505470459519,
 'manufacturer part number': 36.252735229759296,
 'material': 34.9070021881838,
 'model': 23.681619256017505,
 'shoe category': 24.376367614879648,
 'shoe size': 23.326039387308533,
 'size': 21.422319474835884}

In [0]:
df['feat_brand_cat'] = df['feat_brand'].factorize()[0]
df['feat_color_cat'] = df['feat_color'].factorize()[0]
df['feat_gender_cat'] = df['feat_gender'].factorize()[0]
df['feat_manufacturer part number_cat'] = df['feat_manufacturer part number'].factorize()[0]
df['feat_material_cat'] = df['feat_material'].factorize()[0]

df['feat_sport_cat'] = df['feat_sport'].factorize()[0]
df['feat_style_cat'] = df['feat_style'].factorize()[0]

for key in keys:
  df[get_name_feat(key) + '_cat'] = df[get_name_feat(key)].factorize()[0]


In [0]:
feats_cat = [x for x in df.columns if '_cat' in x]

In [68]:
df.weight.unique()

array([nan, '3.0 lbs', '9 g', '1.45 lbs', '0.45 lbs', '1.0 lbs',
       '0.23 lbs', '5.0 lbs', '5.5 lbs', '7.45 lbs', '4.0 lbs',
       '2.7969 lbs', '3.9 lbs', '4.6 pounds', '2.1 lbs', '1.1057 lbs',
       '15.0 lbs', '2.4 ounces', '454 g', '0.105 lbs', '9.1 ounces',
       '4.8 lbs', '6.1 lbs', '6.5 lbs', '1.1041 lbs', '1.3 Kg', '91 g',
       '20.0 lbs', '6.0 lbs', '386 g', '0.81 lbs', '4.5 lbs',
       '0.5 ounces', '2.0 lbs', '3.13 lbs', '5.9 lbs', '6.15 lbs',
       '1 pounds', '1.95 lbs', '2.15 lbs', '2 pounds', '2.1 pounds',
       '14 Kg', '0.4788 lbs', '10.0 lbs', '0.38 lbs', '2.5 lbs',
       '68.912 lbs', '45 g', '13.09 lbs', '2.5 pounds', '0.21 lbs',
       '16.75 lbs', '6.3 lbs', '272 g', '1.8 Kg', '2.8 pounds', '0.1 lbs',
       '5.05 lbs', '0.28 lbs', '76.08 lbs', '0.15 lbs', '200 g',
       '7.8 pounds', '399 g', '4.95 lbs', '64.144 lbs', '24 pounds',
       '73.696 lbs', '1.6 lbs', '6.6 ounces', '5 g', '1.2 Kg', '862 g',
       '3.05 lb', '8.6 ounces', '3.6 lbs', '71.

In [0]:
def convert_weight_to_g(w):
  if not isinstance(w, str):
    return -1

  unit_w = w.split(" ")[-1]
  number_w = float(w.split(" ")[0])

  if unit_w == "pounds" or unit_w == "lb" or unit_w == "lbs":
    return int(number_w * 454)
  elif unit_w == "g":
    return int(number_w)
  elif unit_w == "Kg":
    return int(number_w * 1000)
  elif unit_w == "ounces":
    return int(number_w * 28.3)
  else: return -1


In [0]:
df['weight_int'] = df['weight'].map(convert_weight_to_g)
df[['weight_int', 'weight']].sample(40)

In [0]:
feats = ['brand_cat', 'feat_brand_cat', 'feat_color_cat', 'feat_gender_cat', 'feat_material_cat', 'feat_sport_cat', 'feat_style_cat', 'feat_metal type_cat', 'feat_size_cat', 'feat_occasion_cat', 'weight_int']
model = RandomForestRegressor(max_depth=24, n_estimators=800, min_samples_split=4, min_samples_leaf=2)
result = run_model(feats, model)

# model = RandomForestRegressor(max_depth=30, n_estimators=500, min_samples_split=5, min_samples_leaf=3) => (-46.10563221499396, 4.4808955193906215)
# model = RandomForestRegressor(max_depth=30, n_estimators=500, min_samples_split=5, min_samples_leaf=2) => (-45.823171188074774, 4.395939117513523)

**Sprawdzamych jakość wybranych cech**


In [128]:
X = df[feats].values
y = df['prices_amountmin'].values

m = RandomForestRegressor(max_depth=24, n_estimators=800, min_samples_split=4, min_samples_leaf=2)
m.fit(X, y)

print(result)
perm = PermutationImportance(m, random_state=1).fit(X, y);
#print(perm)
eli5.show_weights(perm, feature_names=feats)

(-45.95101650154759, 4.466736950239678)


Weight,Feature
0.7716 ± 0.0083,brand_cat
0.2595 ± 0.0092,feat_material_cat
0.1288 ± 0.0076,feat_brand_cat
0.1222 ± 0.0074,feat_color_cat
0.1121 ± 0.0037,feat_gender_cat
0.0881 ± 0.0040,feat_style_cat
0.0277 ± 0.0020,feat_size_cat
0.0233 ± 0.0030,weight_int
0.0124 ± 0.0011,feat_metal type_cat
0.0111 ± 0.0017,feat_occasion_cat
